## Проверка однородности. Независимые выборки

Посмотрим, как применять критерии однородности для независимых выборок. Все критерии, которые мы изучали, уже реализованы в Python.

In [ ]:
# как обычно, начнем с импорта нужных библиотек
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# и зафиксируем seed
np.random.seed(123)

В файле **aggress.xls** содержаться бальные оценки агресивности детей 4-х лет, проявленной в течении 15-ти минут после игры. Первые 12 детей — мальчики, остальные 12 детей — девочки. Необходимо проверить гипотезу однородности мальчиков и девочек.

In [ ]:
# сначала импортируем данные

data = pd.read_excel('data/aggress.xls')

In [ ]:
# посмотрим на них 

data

In [ ]:
# Перед анализом данных полезно визулизировать данные. 
# Это позволит получить какой-то априорный вывод о данных и избежать ошибок в дальнейшем 
# (например, при вызове функций с неправильными альтернативами). 
#
# Для визуализации различий лучше всего использовать "ящик с усами" (boxplot)

data.boxplot(column='AGGRESSN', by='GENDER',figsize=(8,5))
plt.show()

In [ ]:
# Теперь извлечем разделим данные для мальчиков и девочек.
# Преобразовывать в numpy array здесь вовсе не обязательно (это удобно только при оценке параметра сдвига).

sample1 = data[data['GENDER'] == 'BOYS']['AGGRESSN'].to_numpy()
sample2 = data[data['GENDER'] == 'GIRLS']['AGGRESSN'].to_numpy()

### Критерий Колмогорова-Смирнова

Начнем с критерия Колмогорова-Смирнова. Этот критерий проверяет простую гипотезу о равенстве двух (непрерывных!) функций распределений, из которых получены выборки, против любой альтернативы. Он реализован с помощью функции ks_2samp в модуле stats:

* **stats.ks_2samp(sample1, sample2, alternative)** — тут sample1 и sample2 — реализации первой и второй выборки, а alternative — альтернатива, которая используется. По умолчанию alternative=‘two-sided’; другие варианты: ‘less’ и ‘greater’. 

Обратите внимание, что альтернатива формулируется для функций распределений, то есть если мы хотим проверить одностороннюю гипотезу о том, что вторая выборка стохастически принимает большие значения, необходимо использовать alternative=‘less’.

In [ ]:
stats.ks_2samp(sample1,sample2)

In [ ]:
stats.ks_2samp(sample1,sample2, alternative='less')

In [ ]:
stats.ks_2samp(sample1,sample2, alternative='greater')

А можно ли применять критерий Колмогорова-Смирнова для этих данных? Ответ: скорее нет, чем да (данные дискретны).

### Критерий Манна-Уитни

Перейдем к критерию Манна-Уитни. Этот критерий проверяет простую гипотезу о равенстве двух функций распределений, из которых получены выборки, против альтернативы доминирования (!). Он реализован с помощью функции mannwhitneyu в модуле stats:

* **stats.mannwhitneyu(sample1, sample2, alternative)** — тут sample1 и sample2 — реализации первой и второй выборки, а alternative — альтернатива, которая используется. По умолчанию alternative=None; другие варианты: ‘two-sided’, ‘less’ и ‘greater’. 

Официальная документация говорит, что alternative=None оставлена только для совместимости со старыми версиями реализации этого критерия.

Правильно, конечно, использовать односторонние версии критерия Манна-Уитни. Обратите внимание, что в отличие от критерия Колмогорова, тут альтернатива формулируется уже для выборок, то есть если мы хотим проверить одностороннюю гипотезу о том, что вторая выборка стохастически принимает большие значения, необходимо использовать alternative=‘greater’.

In [ ]:
stats.mannwhitneyu(sample1,sample2)

In [ ]:
stats.mannwhitneyu(sample1,sample2, alternative='two-sided')

In [ ]:
stats.mannwhitneyu(sample1,sample2, alternative='greater')

In [ ]:
stats.mannwhitneyu(sample1,sample2, alternative='less')

### Критерий Стьюдента ($t$-тест)

Мы еще изучали параметрические критерии Стьюдента для независимых выборок (Z-критерий и t-критерий). Эти критерии проверяют простую гипотезу о равенстве средних двух нормальных распределений против любой альтернативы. Z-критерий мы не будем рассматривать, так как для него необходимо знать дисперсию (которая никогда не является известной). А t-критерий частично реализован с помощью функции ttest_ind:

* **stats.ttest_ind(sample1, sample2, equal_var)** — t-критерий для случая независимых выборок. Тут sample1 и sample2 — реализации первой и второй выборки, а equal_var булевая переменная, которая связана с предположением равенства дисперсии. По умолчанию она принимает значение True, но лучше использовать False, чтобы запускался аппроксимационный критерий (Уэлча), а не точный (Стьюдента).

Различные альтернативы, к сожалению, пока не реализованы. 

Конечно, перед применением критерия Стьюдента необходимо проверить нормальность данных.

In [ ]:
# посмотрим на гистограмму

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15,4)) 

ax1.hist(sample1, bins=5)
ax2.hist(sample2, bins=5)

plt.show()

In [ ]:
# посмотрим на qqplot

from statsmodels.graphics.gofplots import qqplot # импортируем функцию qqplot

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15,4)) 

qqplot(sample1, line='r', ax=ax1)
qqplot(sample2, line='r', ax=ax2)

plt.show()

Конечно, ни о какой нормальности в этих данный и не идет речь. Тем не менее попробуем применить критерий Стьюдента.

In [ ]:
stats.ttest_ind(sample1, sample2)

### Оценка параметра "сдвига"

С помощью "ящиков с усами" мы заметили, что после игры агресивность мальчиков больше, чем у девочек. С помощью критериев однородности, мы убедились, что эти различия значимы. Теперь давайте ответим на вопрос: а на сколько в среднем агресивность мальчиков больше, чем у девочек? Для этого лучше всего использовать медиану попарных разностей.

In [ ]:
np.median([[sample1[i] - sample2[j] for j in range(len(sample2))] for i in range(len(sample1))]) 